# Extra tools

In this notebook we are going to build a tool that taken as input a medical prescription in pdf,jpg format or a string gives me as output the list of facilities that perform medical analysis.

To extract information from prescriptions we will use OCR technology while to match the words in the prescription with those in the dataset we will use the fuzzywuzzy library.

OCR pdf: https://pypi.org/project/PyPDF2/

OCR jpg: https://www.jaided.ai/easyocr/tutorial/

FuzzyWuzzy: https://pypi.org/project/fuzzywuzzy/

# Preliminary operations

Load libraries and dataset

In [1]:
#pip install  pypdf langchain tika typing fuzzywuzzy pymupdf easyocr

In [2]:
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from tika import parser
from typing import List
import re
from typing import List, Tuple
from fuzzywuzzy import fuzz, process
import fitz
import easyocr
import numpy as np
import gradio as gr


c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
reader = easyocr.Reader(['it'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
dataset = pd.read_csv('https://github.com/SimoneFarallo/public_and_social_services/raw/main/data/final_data_cleaned.csv')
dataset = dataset.drop(dataset.columns[0], axis=1)
dataset.head()

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,POLIAMBULATORIALE PARABIAGO,PARABIAGO,MI,VIA XI FEBBRAIO N. 31,ATS DELLA CITTA' METROPOLITANA DI MILANO,2340.0,Struttura ambulatoriale,0331449633,Info non disponibile,No,14.56
1,ALTRE TRAZIONI CUTANEE DEGLI ARTI,9346,ORTOPEDIA E TRAUMATOLOGIA,ATT. CLINICA,POLIAMBULATORIO OSP. MORBEGNO,MORBEGNO,SO,VIA MORELLI 1,ATS DELLA MONTAGNA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.36
2,STREPTOCOCCO ANTICORPI ANTI ANTISTREPTOLISINA-...,91085,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,POLIAMB. A.O.IST.OSPIT.CREMONA,CREMONA,CR,VIA LARGO PRIORI N.1,ATS DELLA VAL PADANA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,4.85
3,MONITORAGGIO INCRUENTO DELLA SATURAZIONE ARTER...,89655,PNEUMOLOGIA,ATT. CLINICA,POLIAMB. IRCCS MAUGERI,PAVIA,PV,"VIA MAUGERI, 10",ATS DI PAVIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,9.3
4,TEST ANTIGENICI (RAPID DIAGNOSTICS TEST Ag-RDT...,90557,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,OSPEDALE BASSINI - CINISELLO BALSAMO,CINISELLO BALSAMO,MI,VIA GORKI 50,ATS DELLA CITTA' METROPOLITANA DI MILANO,30058.0,Struttura di ricovero e cura,02618311,026127543,No,Info non disponibile


# Automatic OCR Search

Taking the functions created earlier as a reference, we create a function that takes both pdf and png as input, also modify the output to be a dataframe so that it can be filtered later

In [5]:
# function to match  prescriptions with a dataset and create a new dataframe
def automatic_ocr_search(file: str, dataset: pd.DataFrame) -> pd.DataFrame:
    # raise error if file path is not a string
    if not isinstance(file, str):
        raise TypeError("File path must be a string.")
    # raise error if dataset is not a pandas DataFrame
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("Dataset must be a pandas.DataFrame.")

    # if file is a pdf, extract text from file
    if file.endswith(".pdf"):
        pdf_file = fitz.open(file) # open pdf file
        text = ''
        for page in pdf_file:
            text += page.get_text() # extract text from each page

        # identify words of interest using regular expression
        words = re.findall(r' - ([A-Za-z\s]+)', text)

    # if file is a png, read text from image using OCR
    elif file.endswith(".png"):
        ricetta = reader.readtext(file) # use tesseract to read text
        result_string = ""
        for i in range(len(ricetta)):
            elemento = ricetta[i][1] #string values are stored in this position in the tuple
            result_string += elemento # concatenate all strings
            result_string += " " # add space between each string
        
        result_string = result_string.replace(".","") # remove dots
        regex = r"\((\d\w*)\)([A-Z ]+)"
        matches = re.findall(regex, result_string) # identify string patterns using regex
        output = []
        for match in matches:
            output.append(match[1]) # append string to output list

        words = output

    # raise error if file format is not supported
    else:
        raise ValueError("File format not supported. Please use a pdf or png file.")

    # match words with values in Codice prestazione ambulatoriale column of dataset using fuzzywuzzy library
    matches = []
    for word in words:
        if not isinstance(word, str):
            raise TypeError("Word of interest must be a string.")
        scores = process.extract(word, dataset['Codice prestazione ambulatoriale'], scorer=fuzz.token_set_ratio, limit=1)
        filtered_scores = [score for score in scores if score[1] >= 90]
        for score in filtered_scores:
            rows = dataset[dataset['Codice prestazione ambulatoriale'] == score[0]].iterrows()
            matches.extend([(word,row[1], score[1]) for row in rows])

    # sort matches by score in descending order
    matches_sorted = sorted(matches, key=lambda x: x[2], reverse=True)
    
    # create empty dictionary to populate with data
    data = {}

    # iterate through list of tuples
    for row in matches_sorted:
        # extract category name and associated row data
        category, row_data, _ = row
        # iterate through columns in row data
        for col, val in row_data.items():
            # create key in dictionary if it doesn't exist already
            if col not in data:
                data[col] = []
            # append value to list of values associated with key
            data[col].append(val)
        # if a column is missing from current row, add a None value
        for col in data.keys():
            if col not in row_data:
                data[col].append(None)

    # create new DataFrame from dictionary
    df = pd.DataFrame(data)

    return df

In [6]:

cd C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette

C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette


In [7]:
output_ricetta_1_png = automatic_ocr_search('ricetta_1.png', dataset)
output_ricetta_1_png.head(10)

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,POLICLINICO SAN MARCO,OSIO SOTTO,BG,CORSO EUROPA N. 7,ATS DI BERGAMO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,22.88
1,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,SANTA MARIA CENTRO ANALISI CHIMICO CLINICHE,VOBARNO,BS,VIA PROVINCIALE 77/E,ATS DI BRESCIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,22.88
2,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,LABORATORIO ANALISI MEDICHE ALFA SRL,MEDA,MB,C.SO DELLA RESISTENZA 7,ATS DELLA BRIANZA,1139.0,Servizio di Medicina di Laboratorio,036274505,0362343763,Sì,22.88
3,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,LABORATORIO ESAMI CHIMICO CLINICI,BRONI,PV,"P.ZZA CAGNONI,1",ATS DI PAVIA,2087.0,Servizio di Medicina di Laboratorio,038551367,038551367,Sì,22.88
4,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,"""LE TERRAZZE"" (AMBULATORIO)",CUNARDO,VA,"VIA U. FOSCOLO,6/B",ATS DELL'INSUBRIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,22.88
5,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,LAM CENTRO BIOMEDICO,CASSANO D'ADDA,MI,PIAZZALE GOBETTI,ATS DELLA CITTA' METROPOLITANA DI MILANO,168.0,Struttura ambulatoriale,0284980023,0284980023,Sì,22.88
6,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,ISTITUTO EUROPEO DI ONCOLOGIA,MILANO,MI,VIA RIPAMONTI 435,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,22.88
7,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,AMBULATORIO DI BORMIO,BORMIO,SO,VIA AGOI 8,ATS DELLA MONTAGNA,2920.0,Struttura ambulatoriale,0342847344,Info non disponibile,No,22.88
8,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,POLIAMBULATORIO OSP.DI LECCO,LECCO,LC,"VIA DELL'EREMO, 9/11",ATS DELLA BRIANZA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,22.88
9,DIIDROTESTOSTERONE (DHT),90175,CHIMICA CLINICA,LABORATORIO,OSPED. S.S.BENEDETTO GELTRUDE,CUGGIONO,MI,"VIA BADI, 4",ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,22.88


In [8]:
output_ricetta_2_pdf = automatic_ocr_search('ricetta_2.pdf', dataset)
output_ricetta_2_pdf.head(10)

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,HUMANITAS GAVAZZENI,BERGAMO,BG,VIA MAURO GAVAZZENI 21,ATS DI BERGAMO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.41
1,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,IRCCS POLICLINICO S.MATTEO,PAVIA,PV,"VIALE GOLGI, 19",ATS DI PAVIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.41
2,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO DI CREMA,CREMA,CR,"LARGO UGO DOSSENA, 5",ATS DELLA VAL PADANA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.41
3,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMB. OSP. VIGEVANO,VIGEVANO,PV,CORSO MILANO 19,ATS DI PAVIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.41
4,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSPEDALE CROTTA OLTROCCHI VAPRIO D'ADDA,VAPRIO D'ADDA,MI,VIA DON MOLETTA N. 22,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.41
5,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO OSP. CHIAVENNA,CHIAVENNA,SO,VIA CERERIA 4,ATS DELLA MONTAGNA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.41
6,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMB. OSP. MORTARA,MORTARA,PV,STRADA PAVESE 1125,ATS DI PAVIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.41
7,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMB.DIV.BUSTO A.,BUSTO ARSIZIO,VA,"VIA ARNALDO DA BRESCIA, 1",ATS DELL'INSUBRIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.41
8,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,LABORATORIO LAM VOGHERA S.R.L.,VOGHERA,PV,VIA GARIBALDI 107/BIS,ATS DI PAVIA,2209.0,Servizio di Medicina di Laboratorio,038348130,038348130,Sì,11.41
9,ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA),90482,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CENTRO DIAGNOSI E ANALISI SRL,SUZZARA,MN,"VIA SOMBOR, 16/B",ATS DELLA VAL PADANA,1996.0,Struttura ambulatoriale,0376531661,0376507322,Sì,11.41


In [9]:
output_ricetta_3_png = automatic_ocr_search('ricetta_3.png', dataset)
output_ricetta_3_png.head(10)

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSP.CIVILE DESTRA SECCHIA-PIEVE CORIANO,BORGO MANTOVANO,MN,"VIA BUGATTE, 1",ATS DELLA VAL PADANA,30239.0,Struttura di ricovero e cura,0386717111,0386717409,No,3.2
1,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,SERVICE LAB FLEMING RESEARCH,MILANO,MI,VIA BERNARDINO QUARANTA 57,ATS DELLA CITTA' METROPOLITANA DI MILANO,19512.0,Servizio di Medicina di Laboratorio,0292956400,0255230125,Sì,3.2
2,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CENTRO CARDIOLOGICO S.P.A.,MILANO,MI,VIA PAREA 4,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
3,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,PRESIDIO OSPEDALIERO - ASOLA,ASOLA,MN,"PIAZZA 80° FANTERIA, 1",ATS DELLA VAL PADANA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
4,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO OSPEDALE LUINO,LUINO,VA,"VIA FORLANINI,6",ATS DELL'INSUBRIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
5,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO OSP.TREVIGLIO,TREVIGLIO,BG,"PIAZZALE OSPEDALE, 1",ATS DI BERGAMO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
6,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSPEDALE MORELLI - SONDALO,SONDALO,SO,VIA ZUBIANI 33,ATS DELLA MONTAGNA,30904.0,Struttura di ricovero e cura,0342808111,0342802146,No,3.2
7,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIALE PARABIAGO,PARABIAGO,MI,VIA XI FEBBRAIO N. 31,ATS DELLA CITTA' METROPOLITANA DI MILANO,2340.0,Struttura ambulatoriale,0331449633,Info non disponibile,No,3.2
8,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,IRCCS S. RAFFAELE - MILANO,MILANO,MI,"VIA OLGETTINA, 60",ATS DELLA CITTA' METROPOLITANA DI MILANO,30935.0,Struttura di ricovero e cura,0226431,0226433128,Sì,3.2
9,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,IST. CLIN. HUMANITAS - ROZZANO,ROZZANO,MI,"VIA MANZONI, 56",ATS DELLA CITTA' METROPOLITANA DI MILANO,30943.0,Struttura di ricovero e cura,0282241,0282242206,Sì,3.2


In [10]:
output_ricetta_3_pdf = automatic_ocr_search('ricetta_3.pdf', dataset)
output_ricetta_3_pdf.head(10)

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSP.CIVILE DESTRA SECCHIA-PIEVE CORIANO,BORGO MANTOVANO,MN,"VIA BUGATTE, 1",ATS DELLA VAL PADANA,30239.0,Struttura di ricovero e cura,0386717111,0386717409,No,3.2
1,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,SERVICE LAB FLEMING RESEARCH,MILANO,MI,VIA BERNARDINO QUARANTA 57,ATS DELLA CITTA' METROPOLITANA DI MILANO,19512.0,Servizio di Medicina di Laboratorio,0292956400,0255230125,Sì,3.2
2,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CENTRO CARDIOLOGICO S.P.A.,MILANO,MI,VIA PAREA 4,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
3,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,PRESIDIO OSPEDALIERO - ASOLA,ASOLA,MN,"PIAZZA 80° FANTERIA, 1",ATS DELLA VAL PADANA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
4,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO OSPEDALE LUINO,LUINO,VA,"VIA FORLANINI,6",ATS DELL'INSUBRIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
5,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO OSP.TREVIGLIO,TREVIGLIO,BG,"PIAZZALE OSPEDALE, 1",ATS DI BERGAMO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
6,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSPEDALE MORELLI - SONDALO,SONDALO,SO,VIA ZUBIANI 33,ATS DELLA MONTAGNA,30904.0,Struttura di ricovero e cura,0342808111,0342802146,No,3.2
7,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIALE PARABIAGO,PARABIAGO,MI,VIA XI FEBBRAIO N. 31,ATS DELLA CITTA' METROPOLITANA DI MILANO,2340.0,Struttura ambulatoriale,0331449633,Info non disponibile,No,3.2
8,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,IRCCS S. RAFFAELE - MILANO,MILANO,MI,"VIA OLGETTINA, 60",ATS DELLA CITTA' METROPOLITANA DI MILANO,30935.0,Struttura di ricovero e cura,0226431,0226433128,Sì,3.2
9,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,IST. CLIN. HUMANITAS - ROZZANO,ROZZANO,MI,"VIA MANZONI, 56",ATS DELLA CITTA' METROPOLITANA DI MILANO,30943.0,Struttura di ricovero e cura,0282241,0282242206,Sì,3.2


In [11]:
output_ricetta_4_pdf = automatic_ocr_search('ricetta_4.pdf', dataset)
output_ricetta_4_pdf.head(10)

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,POLIAMB. OSP.PESENTI-FENAROLI,ALZANO LOMBARDO,BG,"VIA MAZZINI, 88",ATS DI BERGAMO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,7.75
1,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,OSPEDALE CROTTA OLTROCCHI VAPRIO D'ADDA,VAPRIO D'ADDA,MI,VIA DON MOLETTA N. 22,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,7.75
2,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,POLIAMBULATORIO PONTELAMBRO,PONTE LAMBRO,CO,VIA VERDI 3,ATS DELL'INSUBRIA,2765.0,Struttura ambulatoriale,Info non disponibile,Info non disponibile,No,7.75
3,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,CASA DI CURA ANCELLE DELLA CAR,CREMONA,CR,"VIA ASELLI, 14",ATS DELLA VAL PADANA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,7.75
4,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,POLIAMBULATORIO OSP.BOLOGNINI,SERIATE,BG,VIA PADERNO 21,ATS DI BERGAMO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,7.75
5,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,HABILITA ISTITUTO CLINICO OSPEDALE DI SARNICO,SARNICO,BG,VIA P.A. FACCANONI 6,ATS DI BERGAMO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,7.75
6,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,MULTIMEDICA S.R.L.,MILANO,MI,VIA FANTOLI 16/17,ATS DELLA CITTA' METROPOLITANA DI MILANO,64.0,Servizio di Medicina di Laboratorio,Info non disponibile,Info non disponibile,Sì,7.75
7,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,OSPEDALE CIVILE,LEGNANO,MI,VIA PAPA GIOVANNI PAOLO II,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,7.75
8,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,POLIAMB.OSPEDALE GARBAGNATE,GARBAGNATE MILANESE,MI,VIALE FORLANINI N.95,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,7.75
9,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,AMBULATORIO DI DESENZANO D/G,DESENZANO DEL GARDA,BS,LOCALITA' MONTECROCE,ATS DI BRESCIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,7.75


# Search medical facilities with fuzzy

This function does the same thing as the others, but as input it takes a string 

In [12]:
#Function to search structures directly with performance names
def search_facilities(words):
    matches = []
    data = {}

    for word in words:
        if not isinstance(word, str): #Check if the word is a string, otherwise raise a TypeError
            raise TypeError("The word of interest must be a string.")
        scores = process.extract(word, dataset['Codice prestazione ambulatoriale'], scorer=fuzz.token_set_ratio, limit=1) #Extract matching scores between the word and the dataset
        filtered_scores = [score for score in scores if score[1] >= 70] #Filter scores based on a threshold of 70%
        for score in filtered_scores:
            rows = dataset[dataset['Codice prestazione ambulatoriale'] == score[0]].iterrows() #Get the rows of the dataset that match the word
            matches.extend([(word,row[1], score[1]) for row in rows]) #Extend the matches list with a tuple of the word, the row data, and the matching score
            for row in matches:
                category, row_data, _ = row
                for col, val in row_data.items():
                    if col not in data:
                        data[col] =[]
                    data[col].append(val) #Add the values of therow to the respective column in the data dictionary
                for col in data.keys():
                    if col not in row_data:
                        data[col].append(None) #If a column is missing in the row, add None as a placeholder

    df = pd.DataFrame(data) #Create a pandas DataFrame from the data dictionary
    return df

In [13]:
output_query = search_facilities(['calcitonina','emocromo'])
output_query.head()

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,POLIAMBULATORIALE PARABIAGO,PARABIAGO,MI,VIA XI FEBBRAIO N. 31,ATS DELLA CITTA' METROPOLITANA DI MILANO,2340.0,Struttura ambulatoriale,0331449633,Info non disponibile,No,14.56
1,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,POLIAMB. IRCCS MAUGERI,PAVIA,PV,"VIA MAUGERI, 10",ATS DI PAVIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,14.56
2,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,LABORATORIO M.A.D. ANALISI S.R.L.,VOGHERA,PV,VIA XX SETTEMBRE 82,ATS DI PAVIA,2210.0,Servizio di Medicina di Laboratorio,0383642130,0383361210,Sì,14.56
3,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,LABORATORIO LAM SANT'AMBROGIO SRL - MORTARA,MORTARA,PV,VIA STRADA VECCHIA PER GALLIATE 4,ATS DI PAVIA,2085.0,Servizio di Medicina di Laboratorio,038499720,038499720,Sì,14.56
4,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,"OSPEDALE ""E. MORELLI"" - SONDALO",SONDALO,SO,VIA ZUBIANI 33,ATS DELLA MONTAGNA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,14.56


# Filter dataset

This function allows the dataset to be filtered by city and facility type, ideally other filters can be added

In [14]:
#Function to filter the dataset
#Is possible add more columns to filter, for now we use only 2
def filter_data(data_frame, comune=None, risposta_strutt= None):
    if comune is not None and risposta_strutt is not None:
        filtered_df = data_frame[(data_frame['Comune struttura'] == comune) & (data_frame['Struttura privata'] == risposta_strutt)]
    elif comune is not None:
        filtered_df = data_frame[data_frame['Comune struttura'] == comune]
    elif risposta_strutt is not None:
        filtered_df = data_frame[data_frame['Struttura privata'] == risposta_strutt]
    else:
        filtered_df = data_frame.copy()

    return filtered_df

In [15]:
#Filter for Brescia city
filter_data(output_ricetta_3_png,'BRESCIA')

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
22,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CASA DI CURA DOMUS SALUTIS,BRESCIA,BS,VIA LAZZARETTO N. 3,ATS DI BRESCIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
24,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSPEDALE DEI BAMBINI - BRESCIA,BRESCIA,BS,P.LE SPEDALI CIVILI 1,ATS DI BRESCIA,30167.0,Struttura di ricovero e cura,030399069,030395326,No,3.2
48,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,ISTITUTO CLINICO S.ANNA,BRESCIA,BS,VIA DEL FRANZONE 31,ATS DI BRESCIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
65,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,ISTITUTO CLINICO S. ANNA - BRESCIA,BRESCIA,BS,VIA DEL FRANZONE 31,ATS DI BRESCIA,30176.0,Struttura di ricovero e cura,0303197111,0303197635,Sì,3.2
117,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,PRES.POLIAMB.SPEDALI CIVILI BS,BRESCIA,BS,"P.LE SPEDALI CIVILI, 1",ATS DI BRESCIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
163,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CASA DI CURA S.CAMILLO,BRESCIA,BS,VIA TURATI 44,ATS DI BRESCIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
200,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,IST. CLINICO CITTA' DI BRESCIA,BRESCIA,BS,VIA GUALLA 15,ATS DI BRESCIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
229,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,FONDAZIONE POLIAMBULANZA - BRESCIA,BRESCIA,BS,VIA BISSOLATI N. 57,ATS DI BRESCIA,30295.0,Struttura di ricovero e cura,03035151,0303515351,Sì,3.2
250,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,PRES.OSPEDAL.SPEDALI CIVILI BRESCIA,BRESCIA,BS,P.LE SPEDALI CIVILI 1,ATS DI BRESCIA,30906.0,Struttura di ricovero e cura,03039951,Info non disponibile,No,3.2
265,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CASA DI CURA POLIAMBULANZA,BRESCIA,BS,VIA BISSOLATI N°57,ATS DI BRESCIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2


In [16]:
#Filter for Bergamo city  and private structure
filter_data(output_ricetta_4_pdf,'BERGAMO','Sì')

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
100,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,CENTRO BIOMEDICO BERGAMASCO SRL,BERGAMO,BG,ROTONDA DEI MILLE 3/A,ATS DI BERGAMO,1359.0,Servizio di Medicina di Laboratorio,035222332,035236267,Sì,7.75
195,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,LABORATORIO BIOANALITICO DELTA SRL,BERGAMO,BG,VIA CLARA MAFFEI 14,ATS DI BERGAMO,1252.0,Servizio di Medicina di Laboratorio,035221565,035235848,Sì,7.75


In [17]:
#Filter for Como city and private strcture
filter_data(output_query,'COMO','Sì')

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
546,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSPEDALE VALDUCE,COMO,CO,VIA DANTE ALIGHIERI 11,ATS DELL'INSUBRIA,30030.0,Struttura di ricovero e cura,031324111,031308047,Sì,3.2


# Interface gradio

In [18]:
import gradio as gr

In [19]:
# Define the function to be included in the interface
def automatic_ocr_search_interface(
    file: gr.inputs.File,
    comune: str = None,
    risposta_strutt: str = None
):
    # Get the path to the file
    file_path = file.name

    # Run your original function with the file path (string) and the DataFrame
    df = automatic_ocr_search(file_path, dataset)

    # Filters DataFrame if the filtering parameters are specified
    if comune is not None or risposta_strutt is not None:
        df = filter_data(df, comune, risposta_strutt)

    
    return df

# Define the inputs for the user interface
file_input = gr.inputs.File(label="File")
comune_input = gr.inputs.Textbox(label="Comune")
risposta_strutt_input = gr.inputs.Textbox(label="Struttura privata")

# Define the user interface using your Gradio function and the defined inputs
interface = gr.Interface(
    fn=automatic_ocr_search_interface,
    inputs=[file_input, comune_input, risposta_strutt_input],
    outputs=["dataframe"],
    title="Matching Recipe to Dataset",
    description="Insert here your recipe and discover structure in Lombardy.",
    outputs_labels=["Matched Data", "Filtered Data"]
)

# Run interface
interface.launch(share=True)


c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:349: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:352: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:352: UserWarning: `keep_filename` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:30: UserWarning: `

Running on local URL:  http://127.0.0.1:7865

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app. 

Also please ensure that your antivirus or firewall is not blocking the binary file located at: c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\frpc_windows_amd64_v0.2


In [20]:
#Define the function to be included in the interface
def search_facilities_interface(words):
    try:
        df = search_facilities(words)
        unique_values = df
        return unique_values
    except TypeError as e:
        return str(e)

# Define the user interface using your Gradio function and the defined inputs
iface = gr.Interface(fn=search_facilities_interface, inputs="text", outputs="dataframe")

# Run interface
iface.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
